# Unit 2 Assignment: Building a Mixture of Experts (MoE) Router

**Objective:** Build a Smart Customer Support Router using a Mixture of Experts (MoE) architecture.


## 1. Setup and Installation

First, let's install the required packages.

In [47]:
# Install required packages
!pip install groq python-dotenv -q

## 2. Import Libraries and Load API Key

In [48]:
import os
from groq import Groq
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

print("Groq client initialized successfully!")

Groq client initialized successfully!


## 3. Define Expert Configurations

We'll create different expert personalities using specialized system prompts.

In [49]:
# Model configuration for different experts
MODEL_CONFIG = {
    "technical": {
        "name": "Technical Expert",
        "model": "llama-3.3-70b-versatile",
        "system_prompt": """You are a highly skilled Technical Support Engineer with expertise in programming, 
        debugging, and software development. You provide precise, code-focused solutions. When users report errors:
        1. Identify the root cause
        2. Provide a clear explanation
        3. Offer concrete code solutions with examples
        4. Suggest best practices to prevent similar issues
        Be rigorous, technical, and solution-oriented. Use code snippets when appropriate.""",
        "temperature": 0.7
    },
    "billing": {
        "name": "Billing Expert",
        "model": "llama-3.3-70b-versatile",
        "system_prompt": """You are an empathetic Billing and Payments Specialist. You handle financial inquiries 
        with care and professionalism. When users have billing issues:
        1. Acknowledge their concern with empathy
        2. Explain company policies clearly
        3. Provide step-by-step resolution guidance
        4. Offer alternative solutions when possible
        Be understanding, policy-driven, and customer-focused. Ensure users feel heard and supported.""",
        "temperature": 0.7
    },
    "general": {
        "name": "General Expert",
        "model": "llama-3.3-70b-versatile",
        "system_prompt": """You are a friendly and helpful General Customer Support Representative. 
        You handle casual conversations, general inquiries, and provide information about products and services.
        Be conversational, informative, and helpful. Make users feel welcome and guide them to the right resources.""",
        "temperature": 0.7
    }
}

print("Expert configurations defined!")
print(f"Available experts: {', '.join(MODEL_CONFIG.keys())}")

Expert configurations defined!
Available experts: technical, billing, general


## 4. Build the Router Function

The router classifies user queries into categories: technical, billing, or general.

In [50]:
def route_prompt(user_input: str) -> str:
    """
    Routes the user input to the appropriate expert category.
    
    Args:
        user_input: User's query string
        
    Returns:
        Category name: 'technical', 'billing', or 'general'
    """
    routing_prompt = f"""You are a query classifier for a customer support system.
    
Classify the following user query into EXACTLY ONE of these categories:
- technical: For bug reports, errors, code issues, software problems, technical questions
- billing: For payment issues, refunds, subscriptions, charges, invoices, pricing
- general: For general questions, casual chat, product information, greetings

User Query: "{user_input}"

Return ONLY the category name (technical, billing, or general) without any explanation or punctuation."""
    
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a precise classifier. Return only the category name."},
                {"role": "user", "content": routing_prompt}
            ],
            temperature=0,  # Low temperature for consistent classification
            max_tokens=10
        )
        
        category = response.choices[0].message.content.strip().lower()
        
        # Validate the category
        if category not in MODEL_CONFIG:
            print(f"WARNING: Router returned unexpected category: '{category}', defaulting to 'general'")
            return "general"
        
        return category
    
    except Exception as e:
        print(f"ERROR in routing: {e}")
        return "general"  # Default fallback

print("Router function created!")

Router function created!


## 5. Build the Orchestrator Function

The orchestrator routes queries to the appropriate expert and returns the response.

In [51]:
def process_request(user_input: str, verbose: bool = True) -> str:
    """
    Process a user request by routing to the appropriate expert.
    
    Args:
        user_input: User's query string
        verbose: If True, print routing information
        
    Returns:
        Expert's response to the query
    """
    # Step 1: Route the query
    category = route_prompt(user_input)
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"ROUTING DECISION")
        print(f"{'='*60}")
        print(f"Query: {user_input}")
        print(f"Routed to: {category.upper()} Expert")
        print(f"{'='*60}\n")
    
    # Step 2: Get the expert configuration
    expert_config = MODEL_CONFIG[category]
    
    # Step 3: Call the LLM with the expert's system prompt
    try:
        response = client.chat.completions.create(
            model=expert_config["model"],
            messages=[
                {"role": "system", "content": expert_config["system_prompt"]},
                {"role": "user", "content": user_input}
            ],
            temperature=expert_config["temperature"],
            max_tokens=500
        )
        
        answer = response.choices[0].message.content
        
        if verbose:
            print(f"{expert_config['name']} Response:")
            print(f"{'-'*60}")
            print(answer)
            print(f"{'='*60}\n")
        
        return answer
    
    except Exception as e:
        error_msg = f"ERROR: Error processing request: {e}"
        print(error_msg)
        return error_msg

print("Orchestrator function created!")

Orchestrator function created!


## 6. Test the MoE Router System

Let's test our system with different types of queries.

### Test 1: Technical Query

In [52]:
# Technical query test
query1 = "My python script is throwing an IndexError on line 5."
response1 = process_request(query1)


ROUTING DECISION
Query: My python script is throwing an IndexError on line 5.
Routed to: TECHNICAL Expert

Technical Expert Response:
------------------------------------------------------------
# Step-by-step analysis of the problem:
1. **Understanding the Error**: An `IndexError` in Python occurs when you try to access an element in a sequence (like a list or tuple) using an index that does not exist.
2. **Locating the Issue**: The error is happening on line 5 of your script, which implies that there's an indexing operation on that line that's going out of bounds.
3. **Possible Causes**: The most common reasons for this error include:
    - Attempting to access an index that is greater than or equal to the length of the list.
    - Trying to access a negative index that is less than the negative length of the list.
    - Incorrect assumptions about the size or structure of the data being indexed.

# Fixed solution:
Without seeing the actual code, it's challenging to provide a precis

### Test 2: Billing Query

In [53]:
# Billing query test
query2 = "I was charged twice for my subscription this month."
response2 = process_request(query2)


ROUTING DECISION
Query: I was charged twice for my subscription this month.
Routed to: BILLING Expert

Billing Expert Response:
------------------------------------------------------------
I'm so sorry to hear that you were charged twice for your subscription this month. I can imagine how frustrating that must be for you. Please know that I'm here to help and support you in resolving this issue as quickly as possible.

Our company policy is to ensure that all transactions are processed accurately and efficiently. In cases like this, we take immediate action to rectify the situation. I'd like to guide you through the steps to resolve this issue.

To start, I'll need to investigate this further. Can you please provide me with your subscription details, including the date of the duplicate charge and the amount that was deducted? This will help me to look into the matter and determine the cause of the error.

Once I have this information, I'll work with our finance team to process a refun

### Test 3: General Query

In [54]:
# General query test
query3 = "Hello! Can you tell me more about your products?"
response3 = process_request(query3)


ROUTING DECISION
Query: Hello! Can you tell me more about your products?
Routed to: GENERAL Expert

General Expert Response:
------------------------------------------------------------
Hello! I'm so glad you're interested in learning more about our products. We offer a wide range of items across various categories, so I'd be happy to give you an overview.

We have everything from electronics and home appliances to clothing and accessories. Our products are designed to make your life easier, more convenient, and more enjoyable. We're always innovating and updating our product lines to keep up with the latest trends and technologies.

Some of our most popular products include smart home devices, high-quality kitchen appliances, and stylish clothing items. We also have a great selection of gifts and novelty items, perfect for special occasions or just to treat yourself.

If you have something specific in mind, feel free to let me know and I can give you more detailed information about o

### Test 4: More Technical Queries

In [55]:
# Another technical query
query4 = "My React app shows 'Cannot read property of undefined' error in the console."
response4 = process_request(query4)


ROUTING DECISION
Query: My React app shows 'Cannot read property of undefined' error in the console.
Routed to: TECHNICAL Expert

Technical Expert Response:
------------------------------------------------------------
# Step-by-step analysis of the problem:
1. **Understanding the Error**: The "Cannot read property of undefined" error typically occurs when your code is trying to access a property or method of an object that doesn't exist or is undefined.
2. **Possible Causes**: This error can be caused by several factors, including:
   - Trying to access a property of an object before it has been initialized or loaded.
   - Trying to access a property of an object that has been deleted or set to null.
   - Incorrectly assuming that a variable or object has been initialized or passed as a prop.
3. **Identifying the Root Cause**: To identify the root cause, you need to examine the stack trace provided in the console error message. The stack trace will point to the line of code where the 

### Test 5: More Billing Queries

In [56]:
# Another billing query
query5 = "How do I cancel my subscription and get a refund?"
response5 = process_request(query5)


ROUTING DECISION
Query: How do I cancel my subscription and get a refund?
Routed to: BILLING Expert

Billing Expert Response:
------------------------------------------------------------
I'm so sorry to hear that you're looking to cancel your subscription. I completely understand that circumstances can change, and it's essential to have flexibility with your financial commitments. I'm here to help you navigate this process with ease.

To initiate the cancellation and refund process, I'd like to explain our company's policy. We have a standard 30-day money-back guarantee for all our subscriptions. If you're within this timeframe, you're eligible for a full refund. However, if you're outside of this window, we can still assist you in canceling your subscription, but the refund may be subject to our pro-rated refund policy.

To proceed with the cancellation, please follow these steps:

1. Log in to your account on our website and navigate to the "Subscriptions" section.
2. Click on the "

## 7. Interactive Testing

Test the router with your own queries.

In [64]:
# Interactive testing - try your own queries
def test_router_interactive():
    """Interactive testing function for the MoE router"""
    print("\nMoE Router Interactive Test")
    print("Type 'quit' to exit\n")
    
    while True:
        user_query = input("\nEnter your query: ")
        
        if user_query.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
        
        if not user_query.strip():
            print("WARNING: Please enter a valid query.")
            continue
        
        process_request(user_query)

# Uncomment the line below to run interactive mode
test_router_interactive()


MoE Router Interactive Test
Type 'quit' to exit


ROUTING DECISION
Query: I have paid the money twice
Routed to: BILLING Expert

Billing Expert Response:
------------------------------------------------------------
I can imagine how frustrating that must be for you. I'm so sorry to hear that you've been charged twice for the same payment. I'm here to help you resolve this issue as quickly and efficiently as possible.

First, I want to assure you that we take situations like this very seriously and will do our best to rectify the error. Our company policy is to process refunds for duplicate payments within a timely manner.

To proceed with the refund, I'll need to verify some information with you. Could you please provide me with your payment details, including the date and amount of the duplicate payment? Additionally, please confirm your account information so I can locate the transaction in our system.

Once I've verified the information, I'll guide you through the next steps to ini

## 8. Bonus Challenge: Tool Use Expert

Add a tool-use expert that can fetch live data (mocked for this example).

In [58]:
import json
from datetime import datetime

# Mock functions for tool use
def get_bitcoin_price() -> dict:
    """Mock function to get Bitcoin price"""
    # In real implementation, this would call an API like CoinGecko or CoinMarketCap
    return {
        "price": 58327.45,
        "currency": "USD",
        "timestamp": datetime.now().isoformat(),
        "change_24h": 2.3
    }

def get_weather(location: str) -> dict:
    """Mock function to get weather"""
    # In real implementation, this would call a weather API
    return {
        "location": location,
        "temperature": 22,
        "unit": "Celsius",
        "condition": "Partly Cloudy",
        "humidity": 65
    }

# Available tools
AVAILABLE_TOOLS = {
    "get_bitcoin_price": get_bitcoin_price,
    "get_weather": get_weather
}

# Add tool expert to configuration
MODEL_CONFIG["tool"] = {
    "name": "Tool Use Expert",
    "model": "llama-3.3-70b-versatile",
    "system_prompt": """You are a Tool Use Specialist. You can fetch live data from external sources.
    Available tools:
    - get_bitcoin_price: Returns current Bitcoin price
    - get_weather: Returns weather for a location
    
    When presenting data:
    1. Clearly state the information retrieved
    2. Include relevant context and explanations
    3. Be concise and accurate""",
    "temperature": 0.5
}

print("Tool Use Expert configured!")

Tool Use Expert configured!


In [59]:
def route_prompt_with_tools(user_input: str) -> str:
    """
    Enhanced router that includes tool-use category.
    """
    routing_prompt = f"""You are a query classifier for a customer support system.
    
Classify the following user query into EXACTLY ONE of these categories:
- technical: For bug reports, errors, code issues, software problems
- billing: For payment issues, refunds, subscriptions, charges
- tool: For requests about live data like cryptocurrency prices, weather, stock prices
- general: For general questions, casual chat, product information

User Query: "{user_input}"

Return ONLY the category name without any explanation or punctuation."""
    
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a precise classifier. Return only the category name."},
                {"role": "user", "content": routing_prompt}
            ],
            temperature=0,
            max_tokens=10
        )
        
        category = response.choices[0].message.content.strip().lower()
        
        if category not in MODEL_CONFIG:
            print(f"WARNING: Router returned unexpected category: '{category}', defaulting to 'general'")
            return "general"
        
        return category
    
    except Exception as e:
        print(f"ERROR in routing: {e}")
        return "general"

print("Enhanced router with tools created!")

Enhanced router with tools created!


In [60]:
def process_request_with_tools(user_input: str, verbose: bool = True) -> str:
    """
    Enhanced orchestrator that handles tool use.
    """
    # Step 1: Route the query
    category = route_prompt_with_tools(user_input)
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"ROUTING DECISION")
        print(f"{'='*60}")
        print(f"Query: {user_input}")
        print(f"Routed to: {category.upper()} Expert")
        print(f"{'='*60}\n")
    
    expert_config = MODEL_CONFIG[category]
    
    # Step 2: Handle tool use category specially
    if category == "tool":
        # Determine which tool to call
        tool_data = None
        tool_name = None
        
        if "bitcoin" in user_input.lower() or "btc" in user_input.lower():
            tool_name = "get_bitcoin_price"
            tool_data = AVAILABLE_TOOLS[tool_name]()
        elif "weather" in user_input.lower():
            tool_name = "get_weather"
            # Extract location (simplified)
            tool_data = AVAILABLE_TOOLS[tool_name]("New York")
        
        if tool_data:
            if verbose:
                print(f"Tool Called: {tool_name}")
                print(f"Tool Response: {json.dumps(tool_data, indent=2)}\n")
            
            # Format the response using LLM
            enhanced_prompt = f"""User asked: {user_input}
            
I retrieved this data: {json.dumps(tool_data, indent=2)}

Please provide a clear, natural language response to the user's question using this data."""
            
            try:
                response = client.chat.completions.create(
                    model=expert_config["model"],
                    messages=[
                        {"role": "system", "content": expert_config["system_prompt"]},
                        {"role": "user", "content": enhanced_prompt}
                    ],
                    temperature=expert_config["temperature"],
                    max_tokens=300
                )
                
                answer = response.choices[0].message.content
                
                if verbose:
                    print(f"{expert_config['name']} Response:")
                    print(f"{'-'*60}")
                    print(answer)
                    print(f"{'='*60}\n")
                
                return answer
            
            except Exception as e:
                error_msg = f"ERROR: Error processing tool response: {e}"
                print(error_msg)
                return error_msg
    
    # Step 3: For non-tool categories, process normally
    try:
        response = client.chat.completions.create(
            model=expert_config["model"],
            messages=[
                {"role": "system", "content": expert_config["system_prompt"]},
                {"role": "user", "content": user_input}
            ],
            temperature=expert_config["temperature"],
            max_tokens=500
        )
        
        answer = response.choices[0].message.content
        
        if verbose:
            print(f"{expert_config['name']} Response:")
            print(f"{'-'*60}")
            print(answer)
            print(f"{'='*60}\n")
        
        return answer
    
    except Exception as e:
        error_msg = f"ERROR: Error processing request: {e}"
        print(error_msg)
        return error_msg

print("Enhanced orchestrator with tools created!")

Enhanced orchestrator with tools created!


### Test the Tool Use Expert

In [61]:
# Test tool use with Bitcoin price
query_tool1 = "What is the current price of Bitcoin?"
response_tool1 = process_request_with_tools(query_tool1)


ROUTING DECISION
Query: What is the current price of Bitcoin?
Routed to: TOOL Expert

Tool Called: get_bitcoin_price
Tool Response: {
  "price": 58327.45,
  "currency": "USD",
  "timestamp": "2026-02-24T17:42:18.321317",
  "change_24h": 2.3
}

Tool Use Expert Response:
------------------------------------------------------------
The current price of Bitcoin is $58,327.45 USD, as of February 24, 2026, at 17:42:18. This price has increased by 2.3% over the past 24 hours.



In [62]:
# Test tool use with weather
query_tool2 = "What's the weather like?"
response_tool2 = process_request_with_tools(query_tool2)


ROUTING DECISION
Query: What's the weather like?
Routed to: TOOL Expert

Tool Called: get_weather
Tool Response: {
  "location": "New York",
  "temperature": 22,
  "unit": "Celsius",
  "condition": "Partly Cloudy",
  "humidity": 65
}

Tool Use Expert Response:
------------------------------------------------------------
The current weather in New York is partly cloudy with a temperature of 22 degrees Celsius and a humidity level of 65%.



## 9. System Analysis and Summary

Let's analyze how our MoE router performs.

In [63]:
def test_routing_accuracy():
    """
    Test the routing accuracy with various queries.
    """
    test_cases = [
        ("My code has a NullPointerException", "technical"),
        ("I need a refund for double charges", "billing"),
        ("Hello, how are you?", "general"),
        ("What's the Bitcoin price?", "tool"),
        ("Python import error in my script", "technical"),
        ("Cancel my subscription please", "billing"),
        ("Tell me about your company", "general"),
    ]
    
    print("\n" + "="*70)
    print("ROUTING ACCURACY TEST")
    print("="*70)
    
    correct = 0
    total = len(test_cases)
    
    for query, expected in test_cases:
        actual = route_prompt_with_tools(query)
        is_correct = actual == expected
        correct += is_correct
        
        status = "[PASS]" if is_correct else "[FAIL]"
        print(f"{status} Query: '{query}'")
        print(f"   Expected: {expected} | Actual: {actual}")
        print()
    
    accuracy = (correct / total) * 100
    print("="*70)
    print(f"Accuracy: {correct}/{total} ({accuracy:.1f}%)")
    print("="*70)

# Run the accuracy test
test_routing_accuracy()


ROUTING ACCURACY TEST
[PASS] Query: 'My code has a NullPointerException'
   Expected: technical | Actual: technical

[PASS] Query: 'I need a refund for double charges'
   Expected: billing | Actual: billing

[PASS] Query: 'Hello, how are you?'
   Expected: general | Actual: general

[PASS] Query: 'What's the Bitcoin price?'
   Expected: tool | Actual: tool

[PASS] Query: 'Python import error in my script'
   Expected: technical | Actual: technical

[PASS] Query: 'Cancel my subscription please'
   Expected: billing | Actual: billing

[PASS] Query: 'Tell me about your company'
   Expected: general | Actual: general

Accuracy: 7/7 (100.0%)
